In [1]:
#Importing libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
from sklearn.feature_selection import SelectKBest, chi2
from datetime import datetime, timedelta
import random


In [2]:
# Load the dataset
df  = pd.read_csv("luxottica_eyewear_Master.csv", index_col=None)

In [3]:
#displaying top 5 rows
df.head()

,Customer ID,Age,Gender,State,Store Location,Income Level,Date of First Purchase,Last Purchase Date,Type of Eyewear,Brand,...,Customer Segmentation,Complaint History,Product Return Rate,Cross-Sell/Upsell Success Rate,Purchase Channel Loyalty,Lifetime Value,Average Order Value,Feedback,Sales Driver Index,Number of Product Categories Purchased
0,1.0,62.0,Female,GA,Target Optical,High,25-08-2019,15-09-2022,Safety Eyewear,Gucci,...,Inactive,5.0,0.46,0.27,Low,1738.96815,233.902744,Positive,0.51,3.0
1,NaN,65.0,Female,MA,Target Optical,High,17-01-2020,11-05-2023,Safety Eyewear,Prada,...,High-value,7.0,0.72,0.90,Medium,NaN,552.239465,Positive,1.00,2.0
2,3.0,21.0,Female,NV,LensCrafters,Medium,13-10-2015,05-05-2023,Reading Glasses,Michael Kors,...,Inactive,9.0,0.55,0.79,High,9799.82000,833.200000,Positive,0.74,2.0
3,4.0,28.0,Male,NY,LensCrafters,Medium,26-03-2018,NaN,Reading Glasses,Oakley,...,Inactive,0.0,0.12,0.03,NaN,7027.47000,727.390000,Positive,NaN,1.0
4,5.0,21.0,Female,MA,Pearle Vision,High,11-10-2020,29-06-2021,Reading Glasses,Persol,...,NaN,1.0,0.59,0.01,Medium,NaN,772.635501,Negative,0.99,2.0


In [10]:
#renaming the column name
df.rename(columns={'Cross-Sell/Upsell Success Rate': 'Cross_Sell_or_Upsell_Success_Rate'}, inplace=True)



In [11]:
#renaming the column name
df.rename(columns={'Return/Exchange History': 'Return_Or_Exchange_History'}, inplace=True)

In [12]:
#counting the null values of columns
df.isnull().sum()

Customer ID                               5005
Age                                       5005
Gender                                    5005
State                                     5005
Store Location                            5005
Income Level                              5005
Date of First Purchase                    5005
Last Purchase Date                        5005
Type of Eyewear                           5005
Brand                                     5005
Model                                     5005
Price                                     5005
Discount Amount                           5005
Last Interaction Type                     5005
Mode of Purchase                          5005
Return_Or_Exchange_History                5005
Customer Support Interactions             5005
Social Media Engagement                   5005
Referral Source                           5005
Churn                                     5005
Customer Satisfaction                     5005
Product Usage

In [13]:
#checking datatypes of columns
df.dtypes

Customer ID                               float64
Age                                       float64
Gender                                     object
State                                      object
Store Location                             object
Income Level                               object
Date of First Purchase                     object
Last Purchase Date                         object
Type of Eyewear                            object
Brand                                      object
Model                                      object
Price                                     float64
Discount Amount                           float64
Last Interaction Type                      object
Mode of Purchase                           object
Return_Or_Exchange_History                 object
Customer Support Interactions             float64
Social Media Engagement                   float64
Referral Source                            object
Churn                                      object


In [14]:
#droping customer ID column is which is not required
df =df.drop(columns=['Customer ID'])

In [15]:
df.columns

Index(['Age', 'Gender', 'State', 'Store Location', 'Income Level',
       'Date of First Purchase', 'Last Purchase Date', 'Type of Eyewear',
       'Brand', 'Model', 'Price', 'Discount Amount', 'Last Interaction Type',
       'Mode of Purchase', 'Return_Or_Exchange_History',
       'Customer Support Interactions', 'Social Media Engagement',
       'Referral Source', 'Churn', 'Customer Satisfaction', 'Product Usage',
       'Loyalty Program Participation', 'Engagement with Promotions',
       'Purchase Frequency', 'Subscription Status', 'Customer Segmentation',
       'Complaint History', 'Product Return Rate',
       'Cross_Sell_or_Upsell_Success_Rate', 'Purchase Channel Loyalty',
       'Lifetime Value', 'Average Order Value', 'Feedback',
       'Sales Driver Index', 'Number of Product Categories Purchased'],
      dtype='object')

In [16]:
#rows which have null values
rows_with_nulls = df[df.isnull().any(axis=1)]

In [17]:
#printing the null values
rows_with_nulls

,Age,Gender,State,Store Location,Income Level,Date of First Purchase,Last Purchase Date,Type of Eyewear,Brand,Model,...,Customer Segmentation,Complaint History,Product Return Rate,Cross_Sell_or_Upsell_Success_Rate,Purchase Channel Loyalty,Lifetime Value,Average Order Value,Feedback,Sales Driver Index,Number of Product Categories Purchased
0,62.0,Female,GA,Target Optical,High,25-08-2019,15-09-2022,Safety Eyewear,Gucci,Gucci 0140,...,Inactive,5.0,0.46,0.27,Low,1738.968150,233.902744,Positive,0.51,3.0
1,65.0,Female,MA,Target Optical,High,17-01-2020,11-05-2023,Safety Eyewear,Prada,Double Bridge,...,High-value,7.0,0.72,0.90,Medium,NaN,552.239465,Positive,1.00,2.0
2,21.0,Female,NV,LensCrafters,Medium,13-10-2015,05-05-2023,Reading Glasses,Michael Kors,MK2056,...,Inactive,9.0,0.55,0.79,High,9799.820000,833.200000,Positive,0.74,2.0
3,28.0,Male,NY,LensCrafters,Medium,26-03-2018,NaN,Reading Glasses,Oakley,NaN,...,Inactive,0.0,0.12,0.03,NaN,7027.470000,727.390000,Positive,NaN,1.0
4,21.0,Female,MA,Pearle Vision,High,11-10-2020,29-06-2021,Reading Glasses,Persol,718,...,NaN,1.0,0.59,0.01,Medium,NaN,772.635501,Negative,0.99,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100102,41.0,Male,NaN,Pearle Vision,Medium,07-04-2016,15-11-2022,Sports Eyewear,Prada,NaN,...,Loyal,7.0,0.81,0.80,Medium,9446.040000,255.330000,Neutral,0.10,2.0
100103,56.0,Female,IL,Sunglass Hut,NaN,14-05-2018,28-04-2022,Eyewear Accessories,Oliver Peoples,OV520,...,High-value,6.0,0.61,0.12,High,11065.039920,314.938230,Neutral,0.69,3.0
100104,31.0,Female,GA,Sunglass Hut,Medium,15-07-2018,12-06-2023,Goggles,Fendi,Fendi 0153,...,High-value,3.0,0.96,0.84,Low,6468.370000,NaN,Neutral,0.31,2.0
100105,26.0,Female,WA,Sunglass Hut,Medium,07-10-2019,19-11-2021,NaN,Michael Kors,MK2062,...,At-risk,6.0,0.54,0.18,Low,2136.218717,685.872736,Positive,0.77,3.0


In [18]:
#Checking number of coulmns and rows
df.shape

(100107, 35)

In [19]:
df['Last Purchase Date'] = pd.to_datetime(df['Last Purchase Date'])
df['Date of First Purchase'] = pd.to_datetime(df['Date of First Purchase'])

C:\Users\AB92922\AppData\Local\Temp\ipykernel_10664\815430468.py:1: UserWarning: Parsing '15-09-2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Last Purchase Date'] = pd.to_datetime(df['Last Purchase Date'])
C:\Users\AB92922\AppData\Local\Temp\ipykernel_10664\815430468.py:1: UserWarning: Parsing '29-06-2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Last Purchase Date'] = pd.to_datetime(df['Last Purchase Date'])
C:\Users\AB92922\AppData\Local\Temp\ipykernel_10664\815430468.py:1: UserWarning: Parsing '24-02-2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Last Purchase Date'] = pd.to_datetime(df['Last Purchase Date'])
C:\Users\AB92922\AppData\Local\Temp\ipykernel_10664\815430468.py:1: UserWarning: Parsing '30-04-2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consis

C:\Users\AB92922\AppData\Local\Temp\ipykernel_10664\815430468.py:2: UserWarning: Parsing '25-08-2019' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Date of First Purchase'] = pd.to_datetime(df['Date of First Purchase'])
C:\Users\AB92922\AppData\Local\Temp\ipykernel_10664\815430468.py:2: UserWarning: Parsing '17-01-2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Date of First Purchase'] = pd.to_datetime(df['Date of First Purchase'])
C:\Users\AB92922\AppData\Local\Temp\ipykernel_10664\815430468.py:2: UserWarning: Parsing '13-10-2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['Date of First Purchase'] = pd.to_datetime(df['Date of First Purchase'])
C:\Users\AB92922\AppData\Local\Temp\ipykernel_10664\815430468.py:2: UserWarning: Parsing '26-03-2018' in DD/MM/YYYY format. Provide format or specify infer_datetim

In [20]:
df.dtypes


Age                                              float64
Gender                                            object
State                                             object
Store Location                                    object
Income Level                                      object
Date of First Purchase                    datetime64[ns]
Last Purchase Date                        datetime64[ns]
Type of Eyewear                                   object
Brand                                             object
Model                                             object
Price                                            float64
Discount Amount                                  float64
Last Interaction Type                             object
Mode of Purchase                                  object
Return_Or_Exchange_History                        object
Customer Support Interactions                    float64
Social Media Engagement                          float64
Referral Source                

In [21]:
# Replace spaces with underscores in column names
df.columns = df.columns.str.replace(' ', '_')

# Check the updated column names
print("Updated Column Names:", df.columns)

Updated Column Names: Index(['Age', 'Gender', 'State', 'Store_Location', 'Income_Level',
       'Date_of_First_Purchase', 'Last_Purchase_Date', 'Type_of_Eyewear',
       'Brand', 'Model', 'Price', 'Discount_Amount', 'Last_Interaction_Type',
       'Mode_of_Purchase', 'Return_Or_Exchange_History',
       'Customer_Support_Interactions', 'Social_Media_Engagement',
       'Referral_Source', 'Churn', 'Customer_Satisfaction', 'Product_Usage',
       'Loyalty_Program_Participation', 'Engagement_with_Promotions',
       'Purchase_Frequency', 'Subscription_Status', 'Customer_Segmentation',
       'Complaint_History', 'Product_Return_Rate',
       'Cross_Sell_or_Upsell_Success_Rate', 'Purchase_Channel_Loyalty',
       'Lifetime_Value', 'Average_Order_Value', 'Feedback',
       'Sales_Driver_Index', 'Number_of_Product_Categories_Purchased'],
      dtype='object')


In [22]:
#Exclude rows with missing target values
df = df[df['Churn'].notna()]

In [23]:
df.shape

(95102, 35)

In [24]:
import pandas as pd

def QuanQualDates(df):
    quan = []
    qual = []
    dates = []
    
    for columnName in df.columns:
        if df[columnName].dtype == 'O':
            qual.append(columnName)
        elif pd.api.types.is_datetime64_any_dtype(df[columnName]):
            dates.append(columnName)
        else:
            quan.append(columnName)
            
    return quan, qual, dates



In [25]:
# Apply the function
quantitative, qualitative, dates = QuanQualDates(df)



In [26]:
print("Quantitative Columns:", quantitative)

Quantitative Columns: ['Age', 'Price', 'Discount_Amount', 'Customer_Support_Interactions', 'Social_Media_Engagement', 'Customer_Satisfaction', 'Purchase_Frequency', 'Complaint_History', 'Product_Return_Rate', 'Cross_Sell_or_Upsell_Success_Rate', 'Lifetime_Value', 'Average_Order_Value', 'Sales_Driver_Index', 'Number_of_Product_Categories_Purchased']


In [27]:
print("Qualitative Columns:", qualitative)

Qualitative Columns: ['Gender', 'State', 'Store_Location', 'Income_Level', 'Type_of_Eyewear', 'Brand', 'Model', 'Last_Interaction_Type', 'Mode_of_Purchase', 'Return_Or_Exchange_History', 'Referral_Source', 'Churn', 'Product_Usage', 'Loyalty_Program_Participation', 'Engagement_with_Promotions', 'Subscription_Status', 'Customer_Segmentation', 'Purchase_Channel_Loyalty', 'Feedback']


In [28]:
print("Date Columns:", dates)

Date Columns: ['Date_of_First_Purchase', 'Last_Purchase_Date']


In [29]:
# Fill missing values for categorical columns with the mode
for quan in quantitative:
    df[quan].fillna(df[quan].mode()[0], inplace=True)

In [30]:
# Fill missing values for numerical columns with the median
for qual in qualitative:
    df[qual].fillna(df[qual].mode()[0], inplace=True)

In [31]:
df[dates]

,Date_of_First_Purchase,Last_Purchase_Date
0,2019-08-25,2022-09-15
2,2015-10-13,2023-05-05
3,2018-03-26,NaT
4,2020-11-10,2021-06-29
5,2019-11-14,2023-02-24
...,...,...
100102,2016-07-04,2022-11-15
100103,2018-05-14,2022-04-28
100104,2018-07-15,2023-12-06
100105,2019-07-10,2021-11-19


In [32]:
df[dates].isnull().sum()

Date_of_First_Purchase    4761
Last_Purchase_Date        4754
dtype: int64

In [33]:
# Verify the changes
print(df.isnull().sum())

Age                                          0
Gender                                       0
State                                        0
Store_Location                               0
Income_Level                                 0
Date_of_First_Purchase                    4761
Last_Purchase_Date                        4754
Type_of_Eyewear                              0
Brand                                        0
Model                                        0
Price                                        0
Discount_Amount                              0
Last_Interaction_Type                        0
Mode_of_Purchase                             0
Return_Or_Exchange_History                   0
Customer_Support_Interactions                0
Social_Media_Engagement                      0
Referral_Source                              0
Churn                                        0
Customer_Satisfaction                        0
Product_Usage                                0
Loyalty_Progr

In [34]:
# Function to generate a random date within a given range
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

# Generate random dates to fill missing values
num_rows = df.shape[0]

#fill missing 'Date of First date' with random dates between 2015-05-01 and 2020-12-31

df['Date_of_First_Purchase'] = df['Date_of_First_Purchase']. apply(lambda x: random_date(datetime(2015,5,1), datetime(2020, 12, 31)) if pd.isnull(x) else x)

df['Last_Purchase_Date'] = df['Last_Purchase_Date']. apply(lambda x: random_date(datetime(2021,12,1), datetime(2023, 12, 31)) if pd.isnull(x) else x)





In [35]:
df[dates].isnull().sum()

Date_of_First_Purchase    0
Last_Purchase_Date        0
dtype: int64

In [36]:
df.isnull().sum()

Age                                       0
Gender                                    0
State                                     0
Store_Location                            0
Income_Level                              0
Date_of_First_Purchase                    0
Last_Purchase_Date                        0
Type_of_Eyewear                           0
Brand                                     0
Model                                     0
Price                                     0
Discount_Amount                           0
Last_Interaction_Type                     0
Mode_of_Purchase                          0
Return_Or_Exchange_History                0
Customer_Support_Interactions             0
Social_Media_Engagement                   0
Referral_Source                           0
Churn                                     0
Customer_Satisfaction                     0
Product_Usage                             0
Loyalty_Program_Participation             0
Engagement_with_Promotions      

In [37]:
#dataframe share after cleaning
df.shape

(95102, 35)

In [38]:

cleaned_df_updated = df.copy()

In [39]:
cleaned_df_updated.head(2)

,Age,Gender,State,Store_Location,Income_Level,Date_of_First_Purchase,Last_Purchase_Date,Type_of_Eyewear,Brand,Model,...,Customer_Segmentation,Complaint_History,Product_Return_Rate,Cross_Sell_or_Upsell_Success_Rate,Purchase_Channel_Loyalty,Lifetime_Value,Average_Order_Value,Feedback,Sales_Driver_Index,Number_of_Product_Categories_Purchased
0,62.0,Female,GA,Target Optical,High,2019-08-25,2022-09-15,Safety Eyewear,Gucci,Gucci 0140,...,Inactive,5.0,0.46,0.27,Low,1738.96815,233.902744,Positive,0.51,3.0
2,21.0,Female,NV,LensCrafters,Medium,2015-10-13,2023-05-05,Reading Glasses,Michael Kors,MK2056,...,Inactive,9.0,0.55,0.79,High,9799.82000,833.200000,Positive,0.74,2.0


In [41]:
# Save the  cleaned_df to a CSV file
cleaned_df_updated.to_csv('cleaned_df_luxottica_churn_updated_0108.csv', index=False)


